# Federated FedProx PyTorch MNIST Tutorial
The only difference between this notebook and Federated_Pytorch_MNIST_Tutorial.ipynb is overriding of the `train_epoch` function in model definition. [See details](#FedProx)


In [1]:
#Install dependencies if not already installed
!pip install torch torchvision ~/repos/openfl-fork

Processing /home/itrushkin/repos/openfl-fork


  Created wheel for openfl: filename=openfl-1.0.1-py3-none-any.whl size=18949940 sha256=145133c35c8212596c0a4d7e9673d18775465742543a3cdc08622a469acb71e5
  Stored in directory: /home/itrushkin/.cache/pip/wheels/ed/ad/82/f26baafab4be32d052fb76a6f56c9a76ce6903cc3eed7bb394
Successfully built openfl
  Attempting uninstall: openfl
    Found existing installation: openfl 1.0.1
    Uninstalling openfl-1.0.1:
      Successfully uninstalled openfl-1.0.1


In [2]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
import torchvision.transforms as transforms
import openfl.native as fx
from openfl.federated import FederatedModel,FederatedDataSet


After importing the required packages, the next step is setting up our openfl workspace. To do this, simply run the `fx.init()` command as follows:

In [3]:
#Setup default workspace, logging, etc.
fx.init('torch_cnn_mnist')

Creating Workspace Directories
Creating Workspace Templates

New workspace directory structure:
workspace
├── plan
│   ├── defaults
│   │   ├── data_loader.yaml
│   │   ├── network.yaml
│   │   ├── tasks_keras.yaml
│   │   ├── collaborator.yaml
│   │   ├── assigner.yaml
│   │   ├── task_runner.yaml
│   │   ├── defaults
│   │   ├── tasks_tensorflow.yaml
│   │   ├── aggregator.yaml
│   │   ├── tasks_torch.yaml
│   │   └── tasks_fast_estimator.yaml
│   ├── plan.yaml
│   ├── data.yaml
│   └── cols.yaml
├── logs
├── final_pytorch_model.h5
├── agg_to_col_two_signed_cert.zip
├── data
│   └── MNIST
│       ├── raw
│       └── processed
├── cert
│   ├── ca
│   │   ├── signing-ca.crt
│   │   ├── root-ca.crt
│   │   ├── signing-ca
│   │   ├── signing-ca.csr
│   │   └── root-ca
│   ├── server
│   │   ├── agg_nnlicv611.inn.intel.com.csr
│   │   ├── agg_nnlicv611.csr
│   │   ├── agg_none.key
│   │   ├── agg_nnlicv611.crt
│   │   ├── agg_none.crt
│   │   ├── agg_nnlicv611.inn.intel.com.key
│   │   ├─

Now we are ready to define our dataset and model to perform federated learning on. The dataset should be composed of a numpy arrayWe start with a simple fully connected model that is trained on the MNIST dataset. 

In [4]:
def one_hot(labels, classes):
    return np.eye(classes)[labels]

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.MNIST(root='./data', train=True,
                                        download=True, transform=transform)

train_images,train_labels = trainset.train_data, np.array(trainset.train_labels)
train_images = torch.from_numpy(np.expand_dims(train_images, axis=1)).float()
train_labels = one_hot(train_labels,10)

validset = torchvision.datasets.MNIST(root='./data', train=False,
                                       download=True, transform=transform)

valid_images,valid_labels = validset.test_data, np.array(validset.test_labels)
valid_images = torch.from_numpy(np.expand_dims(valid_images, axis=1)).float()
valid_labels = one_hot(valid_labels,10)

/home/itrushkin/.virtualenvs/openfl_research/lib/python3.8/site-packages/torchvision/datasets/mnist.py:64: UserWarning: train_data has been renamed data
  warnings.warn("train_data has been renamed data")
/home/itrushkin/.virtualenvs/openfl_research/lib/python3.8/site-packages/torchvision/datasets/mnist.py:54: UserWarning: train_labels has been renamed targets
  warnings.warn("train_labels has been renamed targets")
/home/itrushkin/.virtualenvs/openfl_research/lib/python3.8/site-packages/torchvision/datasets/mnist.py:69: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/home/itrushkin/.virtualenvs/openfl_research/lib/python3.8/site-packages/torchvision/datasets/mnist.py:59: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


# FedProx
 - Before training loop, we remember global model into `snapshot` variable.
 - After the calculation of main loss function, we add a proximal term defined in [FedProx paper](https://arxiv.org/pdf/1812.06127.pdf) to it.

In [5]:
from torch.optim import Optimizer
from torch.optim.optimizer import required

class FedProxOptimizer(Optimizer):
    def __init__(self, params, lr=required, mu=0.0, momentum=0, dampening=0, weight_decay=0, nesterov=False):
        if momentum < 0.0:
            raise ValueError("Invalid momentum value: {}".format(momentum))
        if lr is not required and lr < 0.0:
            raise ValueError("Invalid learning rate: {}".format(lr))
        if weight_decay < 0.0:
            raise ValueError("Invalid weight_decay value: {}".format(weight_decay))
        if mu < 0.0:
            raise ValueError("Invalid mu value: {}".format(mu))
        defaults = dict(lr=lr, weight_decay=weight_decay, mu=mu, momentum=momentum, nesterov=nesterov, dampening=dampening)

        if nesterov and (momentum <= 0 or dampening != 0):
            raise ValueError("Nesterov momentum requires a momentum and zero dampening")

        super(FedProxOptimizer, self).__init__(params, defaults)

    def __setstate__(self, state):
        super(FedProxOptimizer, self).__setstate__(state)
        for group in self.param_groups:
            group.setdefault('nesterov', False)

    @torch.no_grad()
    def step(self, closure=None):
        """Performs a single optimization step.
        Arguments:
            closure (callable, optional): A closure that reevaluates the model
                and returns the loss.
        """
        loss = None
        if closure is not None:
            loss = closure()

        for group in self.param_groups:
            weight_decay = group['weight_decay']
            momentum = group['momentum']
            dampening = group['dampening']
            nesterov = group['nesterov']
            mu = group['mu']
            w_old = group['w_old']
            for p, w_old_p in zip(group['params'], w_old):
                if p.grad is None:
                    continue
                d_p = p.grad
                if weight_decay != 0:
                    d_p = d_p.add(p, alpha=weight_decay)
                if momentum != 0:
                    param_state = self.state[p]
                    if 'momentum_buffer' not in param_state:
                        buf = param_state['momentum_buffer'] = torch.clone(d_p).detach()
                    else:
                        buf = param_state['momentum_buffer']
                        buf.mul_(momentum).add_(d_p, alpha=1 - dampening)
                    if nesterov:
                        d_p = d_p.add(buf, alpha=momentum)
                    else:
                        d_p = buf
                if w_old is not None:
                    d_p.add_(p - w_old_p, alpha=mu)
                p.add_(d_p, alpha=-group['lr'])

        return loss

    def set_old_weights(self, old_weights):
        for param_group in self.param_groups:
            param_group['w_old'] = old_weights

In [6]:
feature_shape = train_images.shape[1]
classes       = 10

fl_data = FederatedDataSet(train_images,train_labels,valid_images,valid_labels,batch_size=32,num_classes=classes)

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, 3)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(16, 32, 3)
        self.fc1 = nn.Linear(32 * 5 * 5, 32)
        self.fc2 = nn.Linear(32, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(x.size(0),-1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return F.log_softmax(x, dim=1)
    
    def train_epoch(self, batch_generator):
        from openfl.federated.task import PyTorchTaskRunner
        self.optimizer.set_old_weights([p for p in self.parameters()])
        return PyTorchTaskRunner.train_epoch(self, batch_generator)

    
optimizer = lambda x: FedProxOptimizer(x, lr=1e-3, mu=0.1)

def cross_entropy(output, target):
    """Binary cross-entropy metric
    """
    return F.binary_cross_entropy_with_logits(input=output,target=target)

In [7]:

#Create a federated model using the pytorch class, lambda optimizer function, and loss function
fl_model = FederatedModel(build_model=Net,optimizer=optimizer,loss_fn=cross_entropy,data_loader=fl_data)

The `FederatedModel` object is a wrapper around your Keras, Tensorflow or PyTorch model that makes it compatible with openfl. It provides built in federated training and validation functions that we will see used below. Using it's `setup` function, collaborator models and datasets can be automatically defined for the experiment. 

In [8]:
collaborator_models = fl_model.setup(num_collaborators=2)
collaborators = {'one':collaborator_models[0],'two':collaborator_models[1]}#, 'three':collaborator_models[2]}

In [9]:
#Original MNIST dataset
print(f'Original training data size: {len(train_images)}')
print(f'Original validation data size: {len(valid_images)}\n')

#Collaborator one's data
print(f'Collaborator one\'s training data size: {len(collaborator_models[0].data_loader.X_train)}')
print(f'Collaborator one\'s validation data size: {len(collaborator_models[0].data_loader.X_valid)}\n')

#Collaborator two's data
print(f'Collaborator two\'s training data size: {len(collaborator_models[1].data_loader.X_train)}')
print(f'Collaborator two\'s validation data size: {len(collaborator_models[1].data_loader.X_valid)}\n')

#Collaborator three's data
#print(f'Collaborator three\'s training data size: {len(collaborator_models[2].data_loader.X_train)}')
#print(f'Collaborator three\'s validation data size: {len(collaborator_models[2].data_loader.X_valid)}')

Original training data size: 60000
Original validation data size: 10000

Collaborator one's training data size: 30000
Collaborator one's validation data size: 5000

Collaborator two's training data size: 30000
Collaborator two's validation data size: 5000



We can see the current plan values by running the `fx.get_plan()` function

In [10]:
 #Get the current values of the plan. Each of these can be overridden
import json
print(json.dumps(fx.get_plan(), indent=4, sort_keys=True))

{
    "aggregator.settings.best_state_path": "save/torch_cnn_mnist_best.pbuf",
    "aggregator.settings.db_store_rounds": 1,
    "aggregator.settings.init_state_path": "save/torch_cnn_mnist_init.pbuf",
    "aggregator.settings.last_state_path": "save/torch_cnn_mnist_last.pbuf",
    "aggregator.settings.rounds_to_train": 10,
    "aggregator.template": "openfl.component.Aggregator",
    "assigner.settings.task_groups": [
        {
            "name": "train_and_validate",
            "percentage": 1.0,
            "tasks": [
                "aggregated_model_validation",
                "train",
                "locally_tuned_model_validation"
            ]
        }
    ],
    "assigner.template": "openfl.component.RandomGroupedAssigner",
    "collaborator.settings.db_store_rounds": 1,
    "collaborator.settings.delta_updates": false,
    "collaborator.settings.opt_treatment": "RESET",
    "collaborator.template": "openfl.component.Collaborator",
    "data_loader.settings.batch_size": 2

Now we are ready to run our experiment. If we want to pass in custom plan settings, we can easily do that with the `override_config` parameter

In [11]:
#Run experiment, return trained FederatedModel
final_fl_model = fx.run_experiment(collaborators,{'aggregator.settings.rounds_to_train':5})

/home/itrushkin/.virtualenvs/openfl_research/lib/python3.8/site-packages/openfl/federated/task/runner_pt.py:277: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:143.)
  new_state[k] = pt.from_numpy(tensor_dict.pop(k)).to(device)
/home/itrushkin/.virtualenvs/openfl_research/lib/python3.8/site-packages/openfl/federated/task/runner_pt.py:102: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  data, target = pt.tensor(data).to(self.device), pt.tensor(

/home/itrushkin/.virtualenvs/openfl_research/lib/python3.8/site-packages/openfl/federated/task/runner_pt.py:451: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  data, target = pt.tensor(data).to(self.device), pt.tensor(


In [12]:
#Save final model
final_fl_model.save_native('final_pytorch_model')